In [4]:
import os
from dotenv import load_dotenv

In [5]:
load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set.")


In [1]:
%pip install openai-agents python-dotenv -qU


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Basic Agent Creation

In [7]:
# Agent creation 
from agents import Agent, Runner

agent = Agent(
    name="Basic Agent",
    instructions = "You are a helpful assistant.",
    model="gpt-4o-mini"
)

result = await Runner.run(agent, 'Hi, How are you?')
result.final_output

"I'm here and ready to help! How about you?"

In [11]:
# Agent 2 example 

joke_agent = Agent(
    name = "Joke Agent",
    instructions = "You are a funny assistant. Your jobs is to tell a funny joke.",
    model = "gpt-4o-mini"
)

joke_agent_result = await Runner.run(joke_agent, 'Tell me a joke')
joke_agent_result.final_output

"Why don't skeletons fight each other? \n\nThey don't have the guts!"

In [15]:
# Agent 3 example

language_translator_agent = Agent(
    name = "Language Translator Agent",
    instructions = "You are a translator. Your job is to translate given text into urdu.",
    model = "gpt-4o-mini"
)

language_translator_agent_result = await Runner.run(language_translator_agent, f"translate this joke: {joke_agent_result.final_output} into urdu")
print(language_translator_agent_result.final_output)

ہڈیاں ایک دوسرے سے کیوں نہیں لڑتیں؟

کیونکہ ان میں حوصلہ نہیں ہوتا!


# Structured output using Pydantic

In [6]:
from agents import Agent, Runner

recipe_agent = Agent(
    name = "Recipe Agent",
    instructions = "You are an expert chef. Your job is to provide a recipe for the given dish in detail like the recipe ingredients instructions how to make time in min for making that food.",
    model = "gpt-4o-mini"
)

result = await Runner.run(recipe_agent, "give me a recipe for chicken biryani")
result.final_output

'### Chicken Biryani Recipe\n\n#### **Ingredients:**\n\n**For the Chicken Marinade:**\n- 1.5 lbs (700g) chicken, cut into pieces (bone-in pieces recommended)\n- 1 cup plain yogurt\n- 2 large onions, thinly sliced\n- 4 cloves garlic, minced\n- 1-inch piece ginger, grated\n- 2-3 green chilies, slit (adjust to taste)\n- 1 tablespoon red chili powder\n- 1 teaspoon cumin powder\n- 1 teaspoon coriander powder\n- 1/2 teaspoon turmeric powder\n- 1 tablespoon garam masala\n- Salt to taste\n- Juice of 1 lemon\n- A handful of fresh mint leaves, chopped\n- A handful of fresh cilantro (coriander leaves), chopped\n\n**For the Biryani:**\n- 2 cups basmati rice\n- 4 cups water (for boiling rice)\n- 4 tablespoons ghee (clarified butter) or cooking oil\n- 1-2 bay leaves\n- 4-5 whole cloves\n- 4-5 green cardamom pods\n- 1-2 cinnamon sticks\n- 1 star anise (optional)\n- Saffron strands (optional, soaked in 2 tablespoons warm milk)\n- Fried onions (for garnish)\n- Additional mint and cilantro for garnish\n

In [7]:
from pydantic import BaseModel
from agents import Agent, Runner

class Recipe(BaseModel):
    name: str
    ingredients: list[str]
    time: str
    instructions: list[str]
    

recipe_agent = Agent(
    name = "Recipe Agent",
    instructions = "You are an expert chef. Your job is to provide a recipe for the given dish in detail like the recipe ingredients instructions how to make time in min for making that food.",
    model = "gpt-4o-mini",
    output_type = Recipe
)

result = await Runner.run(recipe_agent, "give me a recipe for chicken biryani")
result.final_output

Recipe(name='Chicken Biryani', ingredients=['2 cups Basmati rice', '500 grams chicken, cut into pieces', '1 large onion, thinly sliced', '2 tomatoes, chopped', '1/2 cup plain yogurt', '1/4 cup chopped fresh cilantro', '1/4 cup chopped fresh mint', '4 cups water', '3 tablespoons cooking oil or ghee', '2 teaspoons ginger-garlic paste', '1 teaspoon turmeric powder', '2 teaspoons biryani masala', '1 teaspoon cumin seeds', '4-5 green chilies, slit', '3-4 whole cloves', '2-3 green cardamoms', '1 small stick of cinnamon', 'Salt to taste', 'Saffron strands (optional)'], time='60 minutes', instructions=['Rinse the Basmati rice in cold water until the water runs clear. Soak it in water for 30 minutes, then drain.', 'In a large pot, heat oil or ghee over medium heat. Add cumin seeds, cloves, cardamom, and cinnamon stick. Sauté for a minute until fragrant.', 'Add the sliced onions and fry until golden brown.', 'Add the ginger-garlic paste and green chilies. Stir and sauté for 2-3 minutes until raw

In [19]:
# Structured output example
from pydantic import BaseModel

class Recipe(BaseModel):
    name: str
    ingredients: list[str]
    instructions: str
    cooking_time: int
    
recipe_agent = Agent(
    name = "Recipe Agent",
    instructions = "You are a recipe generator. Your job is to generate a recipe.",
    model = "gpt-4o-mini",
    output_type=Recipe
)
recipe_agent_result = await Runner.run(recipe_agent, "Generate a recipe for a biryani.")
print(recipe_agent_result.final_output)



name='Chicken Biryani' ingredients=['2 cups basmati rice', '500 grams chicken, cut into pieces', '1 large onion, thinly sliced', '2 tomatoes, chopped', '1/2 cup plain yogurt', '4 cups water', '4 tablespoons biryani masala', '2 bay leaves', '4 green cardamom pods', '5 cloves', '1 cinnamon stick', '4 tablespoons oil or ghee', 'Salt to taste', 'Fresh cilantro and mint for garnishing', 'Saffron strands (optional)'] instructions='1. Rinse the basmati rice under cold water until the water runs clear. Soak the rice in water for 30 minutes, then drain.  \n2. In a large pot, heat oil or ghee over medium heat. Add onions and sauté until golden brown.  \n3. Add the chicken pieces and cook until they turn white.  \n4. Stir in the chopped tomatoes and cook until soft.  \n5. Add yogurt and biryani masala, mixing well. Cook until the chicken is tender and the oil separates from the mixture.  \n6. In another pot, bring 4 cups of water to a boil. Add soaked and drained rice along with bay leaves, carda

# Function Tool in Agent

In [9]:
# Built in Tool
from agents import Agent, Runner, WebSearchTool

web_search_tool = WebSearchTool()


web_search_agent = Agent(
    name = "Web Search Agent",
    instructions = "You are an expert web searcher. Your job is to search the relevant information from the web based on user query.",
    model = "gpt-4o-mini",
    # tools = [web_search_tool]
)


result = await Runner.run(
    web_search_agent,
    "Who is the current president of the United States?",
)

result.final_output


'As of now, the current president of the United States is Joe Biden. He took office on January 20, 2021.'

In [ ]:
# Custom Tool
from agents import Agent, Runner, function_tool

@function_tool
def get_temperature(city: str) -> str:
    return f"The temperature in {city} is 25 degrees Celsius."


get_tem_agent = Agent(
    name = "Temperature Agent",
    instructions = "You are an expert weather agent. Your job is to provide the temperature of the given city and use the tool to give proper response.",
    model= "gpt-4o-mini",
    tools = [get_temperature],
)

result = await Runner.run(get_tem_agent, "What is the temperature in New York?")
result.final_output

Error getting response: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3u0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. (request_id: req_31e0b5a28925cb472f5a68ca53473cfc)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************3u0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [23]:
from agents import Agent, Runner, function_tool

@function_tool
def getTemperature(city: str) -> str:
    return f"70 Degree Fahrenheit in {city}"

@function_tool
def get_weather(city: str) -> str:
    return f"Sunny in {city}"


weather_agent = Agent(
    name = "Weather Agent",
    instructions = "You are a weather agent. Your job is to get the weather of a city and temperature.",
    model = "gpt-4o-mini",
    tools=[getTemperature, get_weather]
)
weather_agent_result = await Runner.run(weather_agent, "Get the weather and current temperature of Karachi.")
print(weather_agent_result.final_output)

The weather in Karachi is sunny, with a current temperature of 70°F.


# Handoffs 
Handoffs allow an agent to delegate tasks to another agent
This is particularly useful in scenarios where different agents specialize in distinct areas. For example, a customer support app might have agents that each specifically handle tasks like order status, refunds, FAQs, etc.

In [24]:
from agents import Agent, Runner, function_tool
from pydantic import BaseModel

class Tutorial(BaseModel):
    outline: str
    tutorial: str

Tutorial_agent = Agent(
    name = "Tutorial Agent",
     handoff_description = "use for generating tutorials based on given outline and topic",
    instructions = ("You are a tutorial agent. Your job is to create a tutorial for a given topic.", "give detail tutorial on every topic of the outline also add comments explanation and code snippets."),
    model = "gpt-4o-mini",
    output_type = Tutorial
)


outline_agent = Agent(
    name = "Outline Agent",
    instructions = "You are an outline agent. Your job is to create an outline for a given topic.",
    model = "gpt-4o-mini",
    handoffs = [Tutorial_agent]
)

tutorial_response = await Runner.run(outline_agent, "Create an outline for a tutorial on Python lists.")
print(tutorial_response.final_output)


# Outline for a Tutorial on Python Lists

## I. Introduction
   A. What are Lists?
   B. Importance of Lists in Python
   C. Overview of Tutorial Structure

## II. Creating Lists
   A. Syntax for List Creation
   B. Different Ways to Create Lists
      1. Using Square Brackets
      2. Using the `list()` Function
      3. Creating an Empty List
   C. Example: Basic List Creation

## III. Accessing List Elements
   A. Indexing in Python Lists
      1. Positive Indexing
      2. Negative Indexing
   B. Slicing Lists
      1. Basic Slicing
      2. Advanced Slicing Techniques
   C. Example: Accessing Elements in a List

## IV. Modifying Lists
   A. Adding Elements
      1. Using `append()`
      2. Using `insert()`
      3. Using `extend()`
   B. Removing Elements
      1. Using `remove()`
      2. Using `pop()`
      3. Using `clear()`
   C. Example: Modifying List Contents

## V. List Methods and Functions
   A. Common List Methods
      1. `sort()`
      2. `reverse()`
      3. `count(

# Multi AI Agents using handoffs 

In [32]:
from agents import Agent, Runner, function_tool, handoff, RunContextWrapper


math_tutor_agent = Agent(
    name = "Math Tutor Agent",
    handoff_description = "Specialized agent for solving math problems",
    instructions = "You are a math tutor. Your job is to solve math problems.",
    model = "gpt-4o-mini",
)

def math_tutor_handoff(context: RunContextWrapper[None]):
    print('Handing off to math tutor agent')
    

history_tutor_agent = Agent(
    name = "History Tutor Agent",
    handoff_description = "Specialized agent for solving history problems",
    instructions = "You are a history tutor. Your job is to solve history problems.",
    model = "gpt-4o-mini",
)

def history_tutor_handoff(context: RunContextWrapper[None]):
    print('Handing off to history tutor agent')


multi_agent_team = Agent(
    name = "Multi-Agent Team",
    instructions = "You are a multi-agent team. Your job is to solve problems using specialized agents.",
    handoffs=[handoff(math_tutor_agent, on_handoff=math_tutor_handoff), handoff(history_tutor_agent, on_handoff=history_tutor_handoff)],
)

result = await Runner.run(multi_agent_team, "who is the first president of USA")
result.final_output

Handing off to history tutor agent


'The first president of the United States was George Washington. He served from April 30, 1789, to March 4, 1797. Washington is often called the "Father of His Country" for his pivotal role in leading the nation during the founding years.'

# Streaming 

## Raw response events

RawResponsesStreamEvent are raw events passed directly from the LLM. They are in OpenAI Responses API format, which means each event has a type (like response.created, response.output_text.delta, etc) and data. These events are useful if you want to stream response messages to the user as soon as they are generated.

In [39]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model="gpt-4o-mini",
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

# In a Jupyter notebook, use await instead of asyncio.run()
await main()

Sure! Here are five jokes for you:

1. **Why don't scientists trust atoms?**  
   Because they make up everything!

2. **What do you call fake spaghetti?**  
   An impasta!

3. **Why did the scarecrow win an award?**  
   Because he was outstanding in his field!

4. **How do you organize a space party?**  
   You planet!

5. **Why did the bicycle fall over?**  
   Because it was two-tired! 

Hope these made you smile!

# Guardrails 

Guardrails run in parallel to your agents, enabling you to do checks and validations of user input. For example, imagine you have an agent that uses a very smart (and hence slow/expensive) model to help with customer requests. You wouldn't want malicious users to ask the model to help them with their math homework. So, you can run a guardrail with a fast/cheap model. If the guardrail detects malicious usage, it can immediately raise an error, which stops the expensive model from running and saves you time/money.

There are two kinds of guardrails:

1. Input guardrails run on the initial user input
2. Output guardrails run on the final agent output

## Input guardrails

Input guardrails run in 3 steps:

1. First, the guardrail receives the same input passed to the agent.
2. Next, the guardrail function runs to produce a GuardrailFunctionOutput, which is then wrapped in an InputGuardrailResult
3. Finally, we check if .tripwire_triggered is true. If true, an InputGuardrailTripwireTriggered exception is raised, so you can appropriately respond to the user or handle the exception.

In [ ]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)

class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

# We'll use this agent in our guardrail function.
guardrail_agent = Agent( 
    name="Guardrail check",
    instructions="Check if the user is asking you to do their customer support related query or out of context.",
    output_type=MathHomeworkOutput,
)


# This is the guardrail function that receives the agent's input/context, and returns the result.
@input_guardrail
async def math_guardrail( 
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output, 
        tripwire_triggered=result.final_output.is_math_homework,
    )


agent = Agent(  
    name="Customer support agent",
    instructions="you are only design to answer customer support related queries. if user ask me to do math homework, please refuse.",
    input_guardrails=[math_guardrail],
)

async def main():
    # This should trip the guardrail
    try:
        result = await Runner.run(agent, "how to cook biryani")
        print("Guardrail didn't trip - this is unexpected")
        print(result.final_output)

    except InputGuardrailTripwireTriggered:
        print("Math homework guardrail tripped")

await main()

## Output guardrails

Output guardrails run in 3 steps:

1. First, the guardrail receives the same input passed to the agent.
2. Next, the guardrail function runs to produce a GuardrailFunctionOutput, which is then wrapped in an OutputGuardrailResult
3. Finally, we check if .tripwire_triggered is true. If true, an OutputGuardrailTripwireTriggered exception is raised, so you can appropriately respond to the user or handle the exception.

In [52]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)
class MessageOutput(BaseModel): 
    response: str

class MathOutput(BaseModel): 
    reasoning: str
    is_math: bool

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the output includes any math.",
    output_type=MathOutput,
)

@output_guardrail
async def math_guardrail(  
    ctx: RunContextWrapper, agent: Agent, output: MessageOutput
) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, output.response, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )

agent = Agent( 
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[math_guardrail],
    output_type=MessageOutput,
)

async def main():
    # This should trip the guardrail
    try:
        await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
        print("Guardrail didn't trip - this is unexpected")

    except OutputGuardrailTripwireTriggered:
        print("Math output guardrail tripped")

# Multi Conversation AI Agent

In [56]:
from agents import Agent, Runner, TResponseInputItem

simple_agent = Agent(
    name="Simple Agent",
    instructions="You are a helpful assistant.",
    model="gpt-4o-mini",
)

conversation_history: list[TResponseInputItem] = []
print("You are chatting with Alice, Type 'exit' to end the conversation.")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    
    conversation_history.append({"role": "user", "content": user_input})
    print("User:", user_input)
    result = await Runner.run(simple_agent, conversation_history)
    print(f"Alice: {result.final_output}")
    conversation_history = result.to_input_list()

You are chatting with Alice, Type 'exit' to end the conversation.
Goodbye!


# Context Management

In [ ]:
import asyncio
from dataclasses import dataclass

from agents import Agent, RunContextWrapper, Runner, function_tool

@dataclass
class UserInfo:  
    name: str
    uid: int

@function_tool
async def fetch_user_age(wrapper: RunContextWrapper[UserInfo]) -> str:  
    return f"User {wrapper.context.name} is 47 years old"

async def main():
    user_info = UserInfo(name="John", uid=123)

    agent = Agent[UserInfo](  
        name="Assistant",
        tools=[fetch_user_age],
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="What is the age of the user?",
        context=user_info,
    )

    print(result.final_output)  
    # The user John is 47 years old.

if __name__ == "__main__":
    asyncio.run(main())

# Agent Visualization

In [1]:
%pip install "openai-agents[viz]" -qU


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
